**Introduction**

This is a Google Colab Notebook which trains a PySlowFast model (using pytorchvideo) on a dataset of videos. The dataset's directory should have 3 sub-directories, one for "train", "val", and "test". In each of these directories should be more directories, named with the dataset's classes. Then, within each of these directories should be videos, whose label corresponds to the directory they are placed in (e.g. dataset->train->archery->archery_1.mph).

The "Set-up" section is used for downloading necessary packages and intiliazing variables. Take a look at the variables (such as file paths), and change them accordingly.

The "Class Creation for Dataloaders" section creates the classes/objects necessary for creating and using dataloaders later in the process. Within this section is a commented code block which would change the transform function to one of a resnet. Uncommenting this code block, and changing the model to "make_resnet" in "Model Creation" would use a resnet50 instead.

The "Model Creation" section creates the object that is used as the Lightning Module for the model.

The "Training" section trains the model using the Trainer from Pytorch Lightning.

The "Testing" section tests the model using Trainer.test and a script that manually tests every file in the testing dataset.

# Set-up


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext tensorboard

In [ ]:
!pip install pytorchvideo
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=8f5ba7207054e13e1d67561395a2f2d99c56e8309353276b0ebbfa13a4156536
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=db6aa154e15d4b72121974444a766e79ca2894afbfe14e987b8a5eff42cc0a8f
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created whee

In [ ]:
import os
import pytorch_lightning
import pytorchvideo.data
import torch.utils.data
import torchvision
import pytorchvideo
import pytorchvideo.transforms
from torch.nn.functional import softmax
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
import sklearn
#from sklearn.metrics import accuracy_score

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [ ]:
input_dir = '/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1'
checkpoint_path = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/slowfast_r50/not pretrained'
model_name = "slowfast_r50"

num_classes=16
num_labels=11
batch_size = 8
num_workers = 8
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4
clip_duration = (num_frames * sampling_rate)/frames_per_second
device=('cuda' if torch.cuda.is_available() else 'cpu')
pretrained=True
learning_rate=0.0001
dropout_rate = 0.6
gamma = 2

In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

In [ ]:
pred_to_class = {
    0: "triceps",
    1: "lats",
    2: "biceps",
    3: "quads",
    4: "glutes",
    5: "shoulders",
    6: "abs",
    7: "obliques",
    8: "chest",
    9: "lower back",
    10: "hamstrings",
}

id_to_exercise = {
    0: "bench press",
    1: "bicep curl",
    2: "chest fly machine",
    3: "deadlift",
    4: "hip thrust",
    5: "lat pulling",
    6: "lateral raise",
    7: "leg extension",
    8: "leg raises",
    9: "push-up",
    10: "russian twist",
    11: "shoulder press",
    12: "squat",
    13: "t bar row",
    14: "tricep Pushdown",
    15: "tricep dips",
}

class_to_label = {
    0: [8, 5, 0],
    1: [2],
    2: [8],
    3: [4, 9, 10],
    4: [4],
    5: [1, 2],
    6: [5],
    7: [3],
    8: [6],
    9: [8, 5, 0],
    10: [6, 7],
    11: [5],
    12: [3, 4, 10],
    13: [1, 2],
    14: [0],
    15: [0],
}

id_to_label = {
    0: [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    1: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    3: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
    4: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    5: [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    6: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    7: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    8: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    9: [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    10: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0],
    11: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    12: [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    13: [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    14: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    15: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
}

In [ ]:
a = torch.tensor([12, 5, 14, 9, 10, 15, 12, 3])
new_label = torch.tensor([id_to_label[i.item()] for i in a])
print(new_label)

tensor([[0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.]])


# Class Creation for Dataloaders

In [ ]:
class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)


In [ ]:
class VideosDataModule(pytorch_lightning.LightningDataModule):

    # Dataset configuration
    _DATA_PATH = input_dir
    _CLIP_DURATION = clip_duration  # Duration of sampled clip for each video
    _BATCH_SIZE = batch_size
    _NUM_WORKERS = num_workers  # Number of parallel processes fetching data

    def train_dataloader(self):
        #Create the train partition from the list of video labels and video paths
        train_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'train'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=transform
        )

        return torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS
        )

    def val_dataloader(self):
        #Create the validation partition from the list of video labels and video paths
        val_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'val'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=transform
        )

        return torch.utils.data.DataLoader(
            val_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS,
        )

    def test_dataloader(self):
        test_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'test'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=transform
        )

        return torch.utils.data.DataLoader(
            test_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS,
        )

# Model Creation


In [ ]:
from pytorchvideo.models.slowfast import create_slowfast

def make_slowfast():
    return create_slowfast(
        input_channels=(3, 3),
        model_depth=50,
        model_num_class=num_labels,
        norm=nn.BatchNorm3d,
        dropout_rate=dropout_rate,
        slowfast_channel_reduction_ratio=(8, )
    )

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import MultilabelAccuracy, MultilabelAUROC, MultilabelRecall, MultilabelPrecision, MultilabelF1Score

mcaM2 = MultilabelAccuracy(num_labels=num_labels, average='micro')
mcaS = MultilabelAccuracy(num_labels=num_labels, average='none')
roc_auc = MultilabelAUROC(num_labels=num_labels, average="micro")
pre = MultilabelPrecision(num_labels=num_labels, average='macro', threshold=0.5)
rec = MultilabelRecall(num_labels=num_labels, average='macro', threshold=0.5)
f1score = MultilabelF1Score(num_labels=num_labels, average='macro', threshold=0.5)

def post_act(input):
  return softmax(input, dim=1)

class VideoClassificationLightningModule(pytorch_lightning.LightningModule):

    def __init__(self):
        super().__init__()

        self.model = make_slowfast()
        '''
        self.model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)
        self.model.to(device)
        self.model.blocks[6].proj = nn.Linear(in_features=2304, out_features=11, bias=True)
        '''
        self.model.train()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # The model expects a video tensor of shape (B, C, T, H, W), which is the
        # format provided by the dataset
        y_hat = self.model(batch["video"])

        # Compute cross entropy loss, loss.backwards will be called behind the scenes
        # by PyTorchLightning after being returned from this method.

        new_label = torch.tensor([id_to_label[i.item()] for i in batch["label"]])
        loss = F.cross_entropy(y_hat.cpu(), new_label.cpu())

        # Log the train loss to Tensorboard
        self.log("train_loss", loss.item(), batch_size=batch_size)

        accM2 = mcaM2(y_hat.cpu(), new_label.cpu())
        #accS = mcaS(y_hat.cpu(), new_label.cpu())
        self.log("train_accuracy_micro", accM2.item(), batch_size=batch_size)

        print("train_loss:", loss.item(), "train_accuracy_micro:", accM2.item())
        #print("Exercise:", batch["label"], "Unaveraged Accuracy:", accS)

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        new_label = torch.tensor([id_to_label[i.item()] for i in batch["label"]])
        loss = F.cross_entropy(y_hat.cpu(), new_label.cpu())

        self.log("val_loss", loss.item(), batch_size=batch_size)

        accM2 = mcaM2(y_hat.cpu(), new_label.cpu())
        #accS = mcaS(y_hat.cpu(), new_label.cpu())
        self.log("val_accuracy_micro", accM2.item(), batch_size=batch_size)

        print("val_loss:", loss.item(), "val_accuracy_micro:", accM2.item())
        #print("Exercise:", batch["label"], "Unaveraged Accuracy:", accS)

        return loss

    def configure_optimizers(self):
        """
        Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
        usually useful for training video models.
        """
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def test_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        new_label = torch.tensor([id_to_label[i.item()] for i in batch["label"]])
        loss = F.cross_entropy(y_hat.cpu(), new_label.cpu())

        # logs metrics for each testing_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("test_loss", loss.item(), batch_size=batch_size)

        accM2 = mcaM2(y_hat.cpu(), new_label.cpu())
        self.log("test_accuracy_micro", accM2.item(), batch_size=batch_size)

        new_label = new_label.type(torch.IntTensor)

        auc = roc_auc(y_hat.cpu(), new_label.cpu())
        self.log("auc", auc.item(), batch_size=batch_size)

        precision = pre(y_hat.cpu(), new_label.cpu())
        recall = rec(y_hat.cpu(), new_label.cpu())
        f1 = f1score(y_hat.cpu(), new_label.cpu())

        self.log("precision", precision.item(), batch_size=batch_size)
        self.log("recall", recall.item(), batch_size=batch_size)
        self.log("f1", f1.item(), batch_size=batch_size)

        print("test_loss:", loss.item(), "test_accuracy_micro:", accM2.item(), "auc:", auc.item(), "precision:", precision.item(), "recall", recall.item(), "f1", f1.item())

        return loss

# Training

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

early_stopping_callbacks = EarlyStopping(monitor="val_loss", min_delta=0, patience=10, verbose=True, mode="min")

In [ ]:
classification_module = VideoClassificationLightningModule()
data_module = VideosDataModule()
trainer = pytorch_lightning.Trainer(
    default_root_dir=checkpoint_path,
    max_epochs=30,
    accelerator="auto",
    devices="auto",
    strategy='auto',
    enable_checkpointing=True,
    logger=True,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(classification_module, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type | Params
-------------------------------
0 | model | Net  | 62.0 M
-------------------------------
62.0 M    Trainable params
0         Non-trainable params
62.0 M    Total params
248.066   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

val_loss: 4.195748805999756 val_accuracy_micro: 0.4545454680919647
val_loss: 3.9009103775024414 val_accuracy_micro: 0.4886363744735718


Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: ignored

# Testing


In [ ]:
best_checkpoint_path = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/slowfast_r50/not pretrained/lightning_logs/version_0/checkpoints/epoch=29-step=3900.ckpt'

In [ ]:
trainer.test(model=classification_module, datamodule=data_module, ckpt_path=best_checkpoint_path, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/slowfast_r50/not pretrained/lightning_logs/version_0/checkpoints/epoch=29-step=3900.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/slowfast_r50/not pretrained/lightning_logs/version_0/checkpoints/epoch=29-step=3900.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

test_loss: 8.418013572692871 test_accuracy_micro: 0.7045454382896423 auc: 0.6750571727752686 precision: 0.39393943548202515 recall 0.5454546213150024 f1 0.41818177700042725
test_loss: 6.789331912994385 test_accuracy_micro: 0.7386363744735718 auc: 0.820248007774353 precision: 0.5878788232803345 recall 0.7424242496490479 f1 0.5954545736312866
test_loss: 2.1661415100097656 test_accuracy_micro: 0.8863636255264282 auc: 0.9704861640930176 precision: 0.4095238149166107 recall 0.5 f1 0.4312672019004822
test_loss: 4.5091753005981445 test_accuracy_micro: 0.7613636255264282 auc: 0.7968750596046448 precision: 0.45000001788139343 recall 0.539393961429596 f1 0.4675324857234955
test_loss: 6.697253704071045 test_accuracy_micro: 0.6590909361839294 auc: 0.5990867614746094 precision: 0.09177489578723907 recall 0.24242424964904785 f1 0.11363636702299118
test_loss: 4.233832836151123 test_accuracy_micro: 0.7840909361839294 auc: 0.7743589878082275 precision: 0.18636363744735718 recall 0.3030303120613098 f1 0

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            auc            │    0.7915036678314209     │
│            f1             │    0.3343842327594757     │
│         precision         │    0.30899620056152344    │
│          recall           │    0.4228536784648895     │
│    test_accuracy_micro    │     0.759706437587738     │
│         test_loss         │     4.660025119781494     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 4.660025119781494,
  'test_accuracy_micro': 0.759706437587738,
  'auc': 0.7915036678314209,
  'precision': 0.30899620056152344,
  'recall': 0.4228536784648895,
  'f1': 0.3343842327594757}]

In [ ]:
prediction_model = VideoClassificationLightningModule.load_from_checkpoint(best_checkpoint_path)
prediction_model.to(device)
prediction_model.freeze()

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
'''
for param in model.backbone.parameters():
    param.requires_grad = False
'''

In [ ]:
testing_dir = os.path.join(input_dir, "test")
#testing_dir = "/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos"
numSeen = 0
numCorrectTop1 = 0
numCorrectTop5 = 0
classWiseTop1 = {}
classWiseTop5 = {}

for i in pred_to_class:
  classWiseTop1[pred_to_class[i]] = 0
  classWiseTop5[pred_to_class[i]] = 0

for folder in os.listdir(testing_dir):
  new_path = os.path.join(testing_dir, folder)
  for file_name in os.listdir(new_path):
    video_path = os.path.join(new_path, file_name)

    start_sec = 0
    end_sec = start_sec + clip_duration

    video = EncodedVideo.from_path(video_path)

    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

    video_data = transform(video_data)

    inputs = video_data["video"]
    inputs = [i.to(device)[None, ...] for i in inputs]

    preds = prediction_model(inputs)

    preds = post_act(preds)
    pred_classes = preds.topk(k=5).indices

    pred_class_names = [pred_to_class[int(i)] for i in pred_classes[0]]

    numSeen += 1

    if pred_class_names[0] == folder:
        numCorrectTop1 += 1
        classWiseTop1[folder] += 1

    if folder in pred_class_names:
        numCorrectTop5 += 1
        classWiseTop5[folder] += 1

    print("Current Top1:", (numCorrectTop1 / numSeen) * 100, "%")
    print(video_path)
    print(folder)
    print(pred_class_names)

print("Evaluation Finished")
top1accuracy = (numCorrectTop1 / numSeen) * 100
top5accuracy = (numCorrectTop5 / numSeen) * 100
print("Top 1 Accuracy:", top1accuracy, "%")
print("Top 5 Accuracy:", top5accuracy, "%")

Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_57_0.mp4
bench press
['shoulders', 'triceps', 'chest', 'biceps', 'lats']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_57_1.mp4
bench press
['shoulders', 'triceps', 'chest', 'lats', 'biceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_18_0.mp4
bench press
['triceps', 'shoulders', 'chest', 'biceps', 'lats']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_30_0.mp4
bench press
['triceps', 'chest', 'shoulders', 'lats', 'biceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_26_0.mp4
bench press
['triceps', 'chest', 'shoulders', 'biceps', 'lats']


In [ ]:
testing_dir = "/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos"
numSeen = 0
numCorrectTop1 = 0
numCorrectTop5 = 0
classWiseTop1 = {}
classWiseTop5 = {}

for i in pred_to_class:
  classWiseTop1[pred_to_class[i]] = 0
  classWiseTop5[pred_to_class[i]] = 0

for folder in os.listdir(testing_dir):
  new_path = os.path.join(testing_dir, folder)
  for file_name in os.listdir(new_path):
    video_path = os.path.join(new_path, file_name)

    start_sec = 0
    end_sec = start_sec + clip_duration

    video = EncodedVideo.from_path(video_path)

    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

    video_data = transform(video_data)

    inputs = video_data["video"]
    inputs = [i.to(device)[None, ...] for i in inputs]

    preds = prediction_model(inputs)

    preds = post_act(preds)
    pred_classes = preds.topk(k=5).indices

    pred_class_names = [pred_to_class[int(i)] for i in pred_classes[0]]

    numSeen += 1

    if pred_class_names[0] == folder:
        numCorrectTop1 += 1
        classWiseTop1[folder] += 1

    if folder in pred_class_names:
        numCorrectTop5 += 1
        classWiseTop5[folder] += 1

    print("Current Top1:", (numCorrectTop1 / numSeen) * 100, "%")
    print(video_path)
    print(folder)
    print(pred_class_names)

print("Evaluation Finished")
top1accuracy = (numCorrectTop1 / numSeen) * 100
top5accuracy = (numCorrectTop5 / numSeen) * 100
print("Top 1 Accuracy:", top1accuracy, "%")
print("Top 5 Accuracy:", top5accuracy, "%")

Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/20231105_150207.mp4
Personal Test 1
['shoulders', 'lats', 'biceps', 'abs', 'chest']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/spin thing obliques.mp4
Personal Test 1
['abs', 'obliques', 'triceps', 'chest', 'shoulders']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/reverse curl.mp4
Personal Test 1
['biceps', 'abs', 'obliques', 'lats', 'triceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/hammer curl.mp4
Personal Test 1
['biceps', 'lats', 'abs', 'obliques', 'triceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/lunges.mp4
Personal Test 1
['quads', 'hamstrings', 'glutes',